### Scrape the results

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle as pkl
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
path_data = '/content/drive/My Drive/Python_stuff/Bundesliga21/'

In [3]:
teams = {}
teams["Augsburg"] = 1
teams["Bayern"] = 2
teams["Bielefeld"] = 3
teams["Berlin"] = 4
teams["Bochum"] = 5
teams["Dortmund"] = 6
teams["Frankfurt"] = 7
teams["Freiburg"] = 8
teams["Fürth"] = 9
teams["Hoffenheim"] = 10
teams["Köln"] = 11
teams["Leipzig"] = 12
teams["Leverkusen"] = 13
teams["Mainz"] = 14
teams["M'gladbach"] = 15 
teams["Stuttgart"] = 16
teams["Union Berlin"] = 17
teams["Wolfsburg"] = 18

In [ ]:
# Function to scrape results
def scrape():
  filename = path_data + 'results.pkl'
  spieltage = np.full([34,9,4],-1)
  try:
    # open spieltage
    with open(filename,'rb') as f:
      spieltage = pkl.load(f)
  except:
    pass
  start = np.where(spieltage == -1)[0][0]
  print(start)
  for i in range(start,34):
    URL = 'https://www.bundesliga.com/de/bundesliga/spieltag/2021-2022/' + str(i)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    
    spieltag = np.full([9,4],-1)
    list_games = soup.find_all(class_="ng-star-inserted") #matchRow ng-star-inserted
    for j in range(9):
      game = BeautifulSoup(str(list_games[j]), "html.parser")
      game.find_all('a', href=True)
      gam = BeautifulSoup(str(game), "html.parser")
      names = gam.find_all(class_="name d-none d-md-block")
      spieltag[j,0] = teams[names[0].text]
      spieltag[j,3] = teams[names[1].text]
      try:
        spieltag[j,1] = game.find_all(class_="score ng-star-inserted")[0].text
        spieltag[j,2] = game.find_all(class_="score ng-star-inserted")[1].text
      except:
        print('Spieltag ' + str(i) + ', Partie ' + names[0].text + ' vs. ' + names[1].text + ' not played yet!')
    spieltage[i-1] = spieltag
    if sum(spieltage[i-1][:,1]) == -9:
      with open(filename,'wb') as f:
        pkl.dump(spieltage, f)
        print('Stop scraping because the matches are not played.')
      break
  # with open(filename,'wb') as f:
  #   pkl.dump(spieltage, f)
  return spieltage
spieltage = scrape()

In [ ]:
filename = path_data + 'results.pkl'
spieltage = np.full([34,9,4],-1)
with open(filename,'rb') as f:
  spieltage = pkl.load(f)
print(spieltage)

In [4]:
teams = {}
teams["augsburg"] = 1
teams["bayern"] = 2
teams["bielefeld"] = 3
teams["herthabsc"] = 4
teams["vflbochum"] = 5
teams["dortmund"] = 6
teams["frankfurt"] = 7
teams["freiburg"] = 8
teams["fuerth"] = 9
teams["hoffenheim"] = 10
teams["koeln"] = 11
teams["rbleipzig"] = 12
teams["leverkusen"] = 13
teams["mainz"] = 14
teams["mgladbach"] = 15 
teams["stuttgart"] = 16
teams["unionberlin"] = 17
teams["wolfsburg"] = 18

In [ ]:
def scrape_results(spieltag):
  filename = path_data + 'results_table.pkl'
  results_table = np.zeros((34,131,18))
  try:
    # open spieltage
    with open(filename,'rb') as f:
      results_table = pkl.load(f)
  except:
    pass
  for s in range(34):
    if sum(sum(results_table[s])) == 0:
      print('nächster Spieltag: {}'.format(s+1))
      break
  for i in range(s+1,spieltag+1):
    URL = 'https://www.fussballdaten.de/bundesliga/2022/' + str(i) + '/'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    matches = soup.find_all(class_="spiele-row detils")
    print('Spieltag: {}'.format(i))
    for j in range(len(matches)):
      match = BeautifulSoup(str(matches[j]), "html.parser")
      match.find_all(class_="ergebnis")
      link = BeautifulSoup(str(match), "html.parser")
      #link.find('a', href=True)
      links = []
      for a in link.find_all('a', href=True): 
          if a.text: 
              links.append(a['href'])
      # print(links[1])
      heim = links[1].replace('/bundesliga/2022/' + str(i) + '/', '').split('-')[0]
      ausw = links[1].replace('/bundesliga/2022/' + str(i) + '/', '').split('-')[1].replace('/','')
      print('Heimmannschaft: {}'.format(heim))

      teams[heim]
      print('Auswärtsmannschaft: {}'.format(ausw))

      URL = 'https://www.fussballdaten.de' + links[1]
      page = requests.get(URL)
      soup = BeautifulSoup(page.content, "html.parser")
      soup2 = soup.find_all(class_="ergebnis-timeline")
      soup2 = BeautifulSoup(str(soup2), "html.parser")
      time = soup2.find_all(class_="timeline-badge")
      time = [ele.text for ele in time]

      game = np.zeros((131,2)) # 45 + 20, 45 + 20
      game[0][0] = teams[heim]
      game[0][1] = teams[ausw]
      goal = soup2.find_all(class_="timeline-panel")
      goal = [ele for ele in goal if 'class="icon-red-card icon-shadow fs16"' not in str(ele) and 'class="icon-yellow-red-card icon-shadow fs16"' not in str(ele)]
      for k in range(len(goal)):
        # print(time[i][:2],int(time[i].split('+')[i]))
        # print(time[i])
        min = int(time[k].split("'")[0]) # min
        if len(time[k]) > 3:
          overtime = int(time[k].split('+')[1]) # overtime
        else: overtime = 0

        # print(goal[i].text)
        first = goal[k].text.split(':')[0]
        second = goal[k].text.split(':')[1]
        try:
          goa1 = int(first[-2:])
        except:
          goa1 = int(first[-1:])
        # print(goa1)
        try:
          int(second[:2])
          goa2 = int(second[:2])
        except:
          goa2= int(second[:1])
        # print(goa2)
        if min < 45:
          row = min
        elif min == 45:
          row = min + overtime
        elif min > 45 and min < 90:
          row = min + 20
        elif min == 90:
          row = min + overtime + 20

        game[row][0] = goa1
        game[row][1] = goa2
      for k in range(1,130):
        if game[k+1][0] == 0 and game[k+1][1] == 0:
          game[k+1][0] = game[k][0]
          game[k+1][1] = game[k][1]
      if j == 0:
        games = game
      else: games = np.append(games, game, axis=1)
    results_table[i-1] = games
  filename = path_data + 'results_table.pkl'
  with open(filename,'wb') as f:
    pkl.dump(results_table, f)
  # return results_table

In [ ]:
scrape_results(22)